<a href="https://colab.research.google.com/github/moey920/NLP/blob/master/pyLDAvis_%EB%A5%BC_%EC%9D%B4%EC%9A%A9%ED%95%9C_Latent_Dirichlet_Allocation_%EC%8B%9C%EA%B0%81%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# pyLDAvis 를 이용한 Latent Dirichlet Allocation 시각화하기

LDAvis 는 토픽 모델링에 자주 이용되는 Latent Dirichlet Allocation (LDA) 모델의 학습 결과를 시각적으로 표현하는 라이브러리입니다. LDA 는 문서 집합으로부터 토픽 벡터를 학습합니다. 토픽 벡터는 단어로 구성된 확률 벡터, 
P
(
w
|
t
)
 입니다. 토픽 
t
 로부터 단어 
w
 가 발생할 확률을 학습합니다. 토픽 벡터는 bag-of-words model 처럼 고차원 벡터이기 때문에 여러 토픽 간의 관계를 파악하기가 어렵습니다. 또한 각 토픽의 키워드를 인식하기 어렵습니다. LDAvis 는 차원 축소 방법인 Principal Component Analysis (PCA) 와 키워드 추출 방법을 이용하여 토픽 간의 관계와 토픽 키워드를 손쉽게 이해할 수 있도록 도와줍니다. 이번 포스트에서는 Python 라이브러리인 gensim 을 이용하여 LDA 모델을 학습하고, LDAvis 의 Python wrapper 인 pyLDAvis 를 이용하여 시각화를 하는 과정을 살펴봅니다.

## Visualize high dimensional space

고차원의 벡터를 이해하기 위하여 시각화 방법들이 이용됩니다. 대표적인 방법으로 t-SNE 라 불리는 t-Stochastic Neighbor Embedding 이 있습니다. t-SNE 는 고차원 공간에서 유사한 두 벡터가 2 차원 공간에서도 유사하도록, 원 공간에서의 점들 간 유사도를 보존하면서 차원을 축소합니다. 우리가 이해할 수 있는 공간은 2 차원 모니터 (지도) 혹은 3 차원의 공간이기 때문입니다.

In [0]:
from PIL import Image 
im= Image.open('/content/drive/My Drive/tmp/tsne_mnist.png')
im

위 그림은 t-SNE 가 제안되었던 Maaten (2008) 에서 10 개의 숫자 손글씨인 MNIST 데이터를 2 차원으로 압축하여 시각화한 그림입니다. 같은 색은 같은 숫자를 의미합니다. MNIST 는 (28, 28) 크기의 784 차원 데이터입니다. 우리가 784 차원을 상상할 수는 없지만, 이를 2 차원으로 압축하면 어떤 이미지들이 유사한지 시각적으로 이해할 수 있습니다.

이런 목적으로, 딥러닝 모델들을 포함한 여러 머신 러닝 모델들이 학습하는 고차원의 벡터 공간을 이해하기 위한 목적으로 t-SNE 가 이용됩니다. 

t-SNE 외에도 Multi-Dimensional Scaling (MDS) 나 ISOMAP 과 같은 다양한 manifold 알고리즘들이 고차원의 시각화를 위해 이용됩니다. 그리고 더 나아가서는 Deep learning models 들도 시각화를 위해 이용될 수도 있습니다. 아래는 Hinton 교수님의 2006 년도 논문의 그림입니다. 약 2 만 개의 단어로 표현되는 20 News group 문서를 deep belief network 에 학습시켜 얻은 2 차원 벡터입니다. 여기서도 같은 색은 같은 카테고리를 의미합니다.

In [0]:
im2= Image.open('/content/drive/My Drive/tmp/hinton2006.png')
im2

이 방법들 모두 원 공간에서 유사한 벡터가 저차원 공간에서도 유사하기를 기대합니다. 물론 유사도의 metrics 이 다를 수 있습니다. t-SNE 의 경우에는 2 차원의 공간에서 Euclidean distance 를 기준으로 유사하도록 유도합니다. Hinton 교수님의 그림은 벡터 간 내적 (inner product) 이 유사도로 이용됩니다.

고차원의 벡터 시각화에 대한 이야기로 시작한 이유는 토픽 모델링에서 자주 이용되는 Latent Dirichlet Allocation (LDA) 모델이 단어 공간으로 표현되는 토픽 벡터를 학습하기 때문입니다. 뒤이어 LDA 에 대하여 간단히 알아보도록 합니다.

## Brief introduction of Latent Dirichlet Allocation (LDA)

Latent Dirichlet Allocation (LDA) 는 토픽 모델링에 이용되는 대표적인 알고리즘입니다. 여기서 말하는 토픽은 “어떤 주제를 구성하는 단어들”입니다. 추상적인 정의입니다. 흔히 우리가 말하는 “이 글의 주제”와 같습니다. 한 토픽을 설명하기 위하여 특정 단어들이 이용될 것입니다. 문서 집합에서 이 단어 집합을 찾으려는 것이 토픽 모델링입니다. 일종의 word-level semantic clustering 입니다.

LDA 는 세 가지 가정을 합니다. 

첫째, “문서는 여러 개의 토픽을 지닐 수 있고 한 문서는 특정 토픽을 얼마나 지녔는지의 확률 벡터로 표현된다” 입니다. 이 말은 아래와 같은 식으로 기술됩니다. 
t
 는 토픽, 
d
 는 문서입니다.

P
(
t
|
d
)

둘째, “하나의 토픽은 해당 토픽에서 이용되는 단어의 비율로 표현된다” 입니다. 이는 아래와 같은 각 토픽 별 단어의 생성 확률 분포 식으로 표현됩니다. 
w
 은 단어입니다.

P
(
w
|
t
)

그리고 한 문서에서 특정 단어들이 이용될 가능성은 위의 두 확률 분포의 곱으로 표현됩니다. 정확히는 이 사이에 각 토픽이 발생할 확률 
P
(
t
)
 도 곱해집니다. 아래 식은 정확한 LDA 의 식이 아니지만, 개념적인 이해를 위해 아래처럼만 적어두도록 하겠습니다.

∏
i
P
(
w
i
|
t
i
)
⋅
P
(
t
i
|
d
)

사실 LDA 는 Probablistic Latent Semantic Indexing (pLSI) 의 모델의 학습할 패러메터의 개수를 줄여 over-fitting 을 방지하고, 새로운 문서에 대한 topic vector 를 inference 할 수 있도록 개선한 모델입니다. LDA 의 이해는 pLSI 의 이해로부터 시작하는 것이 좋습니다. pLSI 에서는 단어 
w
 와 문서 
d
 가 발생할 확률 
P
(
w
,
d
)
 를 다음처럼 정의합니다. 한 문서가 특정한 토픽 벡터를 지니고, 각 토픽 별 단어 확률 분포와 이를 곱하여 한 문서에서 단어가 발생할 확률을 계산합니다.

P
(
w
,
d
)
=
∑
t
P
(
w
|
t
)
⋅
P
(
t
|
d
)
⋅
P
(
t
)

그러나 우리는 토픽 
t
 의 분포를 모르기 때문에 이를 추정해야 합니다. pLSI 나 LDA 모두 이를 학습합니다. 그 결과 두 모델 모두 
P
(
w
|
t
)
 를 학습합니다. 토픽 
t
 마다 단어 
w
 가 얼마나 자주 등장하는지에 대한 확률 분포입니다.

위 설명은 LDA 와 pLSI 의 간략한 설명입니다. 자세한 LDA 의 설명은 이후 다른 포스트에서 이어 하겠습니다. 기억할 점은, 토픽 모델링은 단어로 표현되는 토픽 벡터를 학습한다는 것 입니다. 그리고 이는 단어 개수만큼의 고차원이며, 이를 시각화하여 토픽 모델링의 결과를 이해하려 합니다.

# Codes

Gensim 은 Python 으로 구현된 topic modeling / embedding 용 라이브러리입니다. Gensim 은 Python 사용자들에게 NLP 의 장벽을 낮춰준 정말 고마운 라이브러리입니다. 처음 0.12 버전까지는 Latent Dirichlet Allocation (LDA), Latent Semantic Indexing (LSI), Random Projection (RP) 와 같은 토픽 모델링 알고리즘들을 제공하였습니다. 이후 Google 에서 공개한 Word2Vec, Doc2Vec 및 Facebook Research 의 FastText, 그리고 keyword & key-sentence extraction 을 위한 TextRank (정확히는 그 변형) 까지 제공하고 있습니다.

pyLDAvis 를 이용하기 위해서는 일단 LDA 를 학습해야 합니다. 우리는 Gensim 을 이용하여 LDA 를 학습하는 방법부터 살펴봅니다.

## Prepare trainable data from text

Gensim 의 공식 홈페이지에서는 LDA 를 학습하는 튜토리얼을 제공하고 있습니다. 이를 간략히 살펴봅니다.

우리는 문서 집합을 가지고 있습니다. 이는 텍스트 파일로, 한 줄이 하나의 문서에 해당합니다. 이 텍스트로부터 LDA 를 학습하기 위한 input data, corpus 를 만듭니다. 토크나이징은 미리 해두었습니다. 각 문서에서 명사만을 남기고, 다른 단어들은 모두 제거한 뒤 뛰어쓰기 기준으로 단어를 구분하였습니다. doc.split() 은 토크나이징 역할을 합니다.

In [0]:
import numpy as np 
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
%matplotlib inline

In [0]:
train_file_link = '/content/drive/My Drive/text/haram_dataset.txt'

In [0]:
# 불러온 데이터를 보면 id, document, label로 구분이 되어있습니다.
train_data = pd.read_csv(train_file_link, header = 0, delimiter = '\t', quoting = 3)
train_data.head(10)

In [0]:
print("불필요 개수: {}".format(train_data['label'].value_counts()[0]))
print("브랜드명 개수 : {}".format(train_data['label'].value_counts()[1]))
print("주소 개수 : {}".format(train_data['label'].value_counts()[2]))
print("사업자등록번호 개수 : {}".format(train_data['label'].value_counts()[3]))
print("구매일시 개수 : {}".format(train_data['label'].value_counts()[4]))
print("상품명 개수 : {}".format(train_data['label'].value_counts()[5]))
print("상품바코드 개수 : {}".format(train_data['label'].value_counts()[6]))
print("결제금액 개수 : {}".format(train_data['label'].value_counts()[7]))
print("카드금액 개수 : {}".format(train_data['label'].value_counts()[8]))
print("카드번호 개수 : {}".format(train_data['label'].value_counts()[9]))
print("카드사명 개수 : {}".format(train_data['label'].value_counts()[10]))
print("카드승인번호 개수 : {}".format(train_data['label'].value_counts()[11]))
print("영수증 바코드 개수 : {}".format(train_data['label'].value_counts()[12]))
print("결제방법 개수 : {}".format(train_data['label'].value_counts()[13]))
print("현금영수증 승인번호 개수 : {}".format(train_data['label'].value_counts()[14]))
print("지점명 개수: {}".format(train_data['label'].value_counts()[15]))
print("지점대표 개수: {}".format(train_data['label'].value_counts()[16]))
print("지점 전화번호 개수: {}".format(train_data['label'].value_counts()[17]))
print("단가 및 금액 개수: {}".format(train_data['label'].value_counts()[18]))
print("구매 수량 개수: {}".format(train_data['label'].value_counts()[19]))


In [0]:
!pip install konlpy

In [0]:
import re
import json
from konlpy.tag import Okt
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer

from tqdm import tqdm

In [0]:
def preprocessing(review, okt, remove_stopwords = False, stop_words = []):
    # 함수의 인자는 다음과 같다.
    # review : 전처리할 텍스트
    # okt : okt 객체를 반복적으로 생성하지 않고 미리 생성후 인자로 받는다.
    # remove_stopword : 불용어를 제거할지 선택 기본값은 False
    # stop_word : 불용어 사전은 사용자가 직접 입력해야함 기본값은 비어있는 리스트
    
    # 1. 한글 및 공백을 제외한 문자 모두 제거. + 영어 소문자, 대문자, 숫자도 제외
    # 일단 OCR 결과의 원형을 학습시키기 위해 정규표현식을 사용하지 않고 학습시켜보겠습니다.
    review_text = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9\\s]", "",  review)
    #review_text = re.sub(" ", "",  review)
    
    # 2. okt 객체를 활용해서 형태소 단위로 나눈다.
    word_review = okt.morphs(review_text, stem=True)
    
    if remove_stopwords:
        
        # 불용어 제거(선택적)
        word_review = [token for token in word_review if not token in stop_words]
        
   
    return word_review

In [0]:
stop_words = ['은', '는', '이', '가', '하', '아', '것', '들','의', '있', '되', '수', '보', 
              '주', '등', '한', '(', ')', '/', '*', '=', 'E', '|', '-', '.', ',', 'II', 'لالالالا', 
              '|||||||||', 'iii', '|||', '. ', '.', '"', ' )', '[', ']']
okt = Okt()
clean_train_review = []

for review in tqdm(train_data['document']):
    # 비어있는 데이터에서 멈추지 않도록 string인 경우만 진행
    if type(review) == str:
        clean_train_review.append(preprocessing(review, okt, remove_stopwords = True, stop_words=stop_words))
    else:
        clean_train_review.append([])  #string이 아니면 비어있는 값 추가

In [0]:
print(clean_train_review[0:9])

이제 토크나이징 된 코퍼스 얻었습니다

In [0]:
corpus_path = clean_train_review

class Documents:
    def __init__(self, path):
        self.path = path
    def __iter__(self):
        with open(self.path, encoding='utf-8') as f:
            for doc in f:
                yield doc.strip().split()

documents = Documents(corpus_path)

In [0]:
corpus_path = clean_train_review

class Documents:
    def __init__(self, documents):
        self.documents = documents
    def __iter__(self):
        with open(self.documents, encoding='utf-8') as f:
            for doc in f:
                yield doc.strip().split()

documents = Documents(corpus_path)

Gensim 의 preprocessing utils 에는 아쉽게도 min count filtering 을 제공하는 vectorizer 가 없습니다. 이를 직접 구현해야 합니다. 일단 단어를 int 형식의 idx 로 변환하는 encoder, Dictioanry 를 학습합니다. Dictionary 에 list of list of str 형식의 documents 를 입력하면 Dictioanry 가 학습됩니다. 아래 예시에서는 총 37,987 개의 단어가 학습되었습니다.

In [0]:
import gensim

dictionary = gensim.corpora.Dictionary(documents)
print('dictionary size : %d' % len(dictionary)) # dictionary size : 37987